In [1]:
import os
from simplified_scrapy import SimplifiedDoc, utils

path = 'data/one_run/m'
#read the whole text from 
myList = []
for root, dirs, files in os.walk(path):
    
    for file in files:
        if file.endswith('osm.net.xml'):
            myList.append(os.path.join(root, file))

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ (e['tl'], e['linkIndex'],e['dir'],e['via']) for e in doc.selects('connection')]

    
arg = list(dict.fromkeys(arg))

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    tl = [ e['id'] for e in doc.selects('tlLogic')]

    


import pandas as pd
data = pd.DataFrame(columns= ["tl","index","dir","via"], index=range(len(arg)))
l = []
for i in range(len(arg)):
    l.append(arg[i][0] )
    if arg[i][0] == tl[0]:
        data.iloc[i,0] = arg[i][0]
        data.iloc[i,3] = arg[i][3]
        data.iloc[i,1] = arg[i][1]
        data.iloc[i,2] = arg[i][2]
        
data.dropna(inplace=True)
data.iloc[:,3] = data.iloc[:,3].str.replace(tl[0],'')
data = data.sort_values(by='index')
data.reset_index(inplace=True,drop=True)
data = data.drop(columns=['tl'])
direction_lane_dict = {}
for i in data.groupby('dir'):
    d = i[0].upper()+'G'
    direction_lane_dict[d] = [int(x) for x in i[1]['index']]
    
for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ (e['tl'],e['from'], e['to'],e['fromLane'],e['toLane']) for e in doc.selects('connection')]
            
arg = list(dict.fromkeys(arg))

import pandas as pd
data = pd.DataFrame(columns= ["from","to","froml","tol"], index=range(len(arg)))
for i in range(len(arg)):
    if arg[i][0] == tl[0]:
        data.iloc[i,0] = arg[i][1]
        data.iloc[i,1] = arg[i][2]
        data.iloc[i,2] = arg[i][3]
        data.iloc[i,3] = arg[i][4]
        
data.dropna(inplace=True)
data['from'] = data['from']+'_'+data['froml']
data['to'] = data['to']+'_'+data['tol'] 
l = list(data['from'].unique())
for i in data['to'].unique():
    l.append(i)

node_light_7 = tl[0]
listLanes = set(l)
#rint(listLanes)

direction_list = [k for k in direction_lane_dict]

for file in myList:
    xml = utils.getFileContent(file)
    doc = SimplifiedDoc(xml)
    arg = [ e['state'] for e in doc.selects('phase')]
         
arg = list(dict.fromkeys(arg))
green = []
lenpahse = len(arg[0])
for i in arg: 
    if 'g' in i or 'G' in i:
        green.append(i)
        
phases_names =[]
phases = {}
controlSignal = []

for i in green:
    p = ''
    for j in range(len(i)):
        if i[j]=='G' or i[j]=='g':
            for key in direction_lane_dict:
                if j+1 in direction_lane_dict[key]:
                    if p == '':
                        p = p + key
                    else:
                        p = p + '_' +key

    if p not in phases_names:
        phases[p] = i
        controlSignal.append(i)
        phases_names.append(p)

phases_light_7 = [p for p in phases]
actions= list(phases.values())
numphases = len(phases_light_7)
Durations=[7,3,7,7,7,7,7,3,7]

In [2]:
import os, sys
import statistics 
#if 'SUMO_HOME' in os.environ:
#    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
#    sys.path.append(tools)
#else:
#    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
from tqdm import tqdm
import random
from time import time

import CH
import PredictPos
import ModifyFile

f_memory = open("memorybasics.csv", "w")
f_memory.write('Information,x,y' + "\n")
f_memory.close()
sumoBinary = "/Users/remi/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary,'-c','data/one_run/m/osm.sumocfg','-n', 'data/one_run/m/osm.net.xml']

time_start = time()
traci.start(sumoCmd)
    
TotalLane = traci.lane.getIDList()
TotalListLane = list(TotalLane)
ListRemove = []

TotalEdge = traci.edge.getIDList()
TotalBigEdge = list(TotalEdge)

for edge in TotalEdge:
    if(traci.edge.getLaneNumber(edge) < 2):
        TotalBigEdge.remove(edge)

allCar = []

for edge in TotalBigEdge:
    for lane in TotalLane:
        if(lane[0:10] in edge):
            ListRemove.append(lane)

ListRemove= list(set(ListRemove))

for lane in TotalLane:
    for lane in ListRemove:
        if lane in TotalListLane:
            TotalListLane.remove(lane)

TotalLane = tuple(TotalListLane)
for lane in TotalLane:
    if(traci.lane.getMaxSpeed(lane)< 8.3):   #8.34 = 30 km/h
        TotalListLane.remove(lane)

count = 2000 #960240
i = 0
while traci.simulation.getTime() <= count:

    traci.simulationStep()
    
    current_time = traci.simulation.getTime()
    
    action= i%9

    duration= Durations[action]    
    traci.trafficlight.setRedYellowGreenState(node_light_7, actions[action])

    Yellow = ''
    for j in range(len(phases_light_7[0])):
        Yellow = Yellow+"y"
        
    for j in range(3):
        timestamp = traci.simulation.getTime()
        traci.trafficlight.setRedYellowGreenState(node_light_7, Yellow)

    memory_time = '\ntime: %d \n' %traci.simulation.getTime()
    f_memory = open("memorybasics.csv", "a")
    f_memory.write(memory_time + "\n")

    for edge in TotalBigEdge:
        listVeh = traci.edge.getLastStepVehicleIDs(edge)
        for vehicle in listVeh:
            if vehicle in allCar:
                pass
            else:
                allCar.append(vehicle)
            position = traci.vehicle.getPosition(vehicle)
            futurPos = PredictPos.FuturPos(vehicle)
            memory_vehi = 'vehicle: %s' %vehicle
            memory_pos = 'Current Position:, %f,%f' %(position[0],position[1])
            memory_fut = 'Predict Position:, %f,%f' %(futurPos[0],futurPos[1])
            f_memory.write(memory_vehi + "\n" + memory_pos + "\n" + memory_fut + "\n")

    for lane in TotalListLane:
        listVeh = traci.lane.getLastStepVehicleIDs(lane)
        for vehicle in listVeh:
            if vehicle in allCar:
                pass
            else:
                allCar.append(vehicle)
            position = traci.vehicle.getPosition(vehicle)
            futurPos = PredictPos.FuturPos(vehicle)
            memory_vehi = 'vehicle: %s' %vehicle
            memory_pos = 'Current Position:, %f,%f' %(position[0],position[1])
            memory_fut = 'Predict Position:, %f,%f' %(futurPos[0],futurPos[1])
            f_memory.write(memory_vehi + "\n" + memory_pos + "\n" + memory_fut + "\n")
    f_memory.close()

    i = i + 1 

print(allCar)
for vehicle in allCar:
    ModifyFile.calculationError(int(vehicle))

ModifyFile.calculationTot()

time_end = time()
print ("traci time elapsed: {}".format(time_end-time_start))
traci.close(True)

 Retrying in 1 seconds
['1', '12', '36', '37', '57', '100', '107', '118', '124', '127', '130', '133', '135', '136', '140', '154', '155', '164', '165', '167', '170', '194', '195', '206', '220', '222', '231', '238', '256', '262', '270', '273', '286', '305', '333', '338', '346', '348', '355', '362', '363', '370', '372', '381', '394', '396', '400', '401', '410', '422', '427', '433', '442', '447', '453', '459', '469', '481', '492', '499', '507', '525', '552', '556', '560', '570', '576', '590', '593', '600', '606', '608', '619', '641', '643', '648', '667', '672', '673', '681', '682', '713', '734', '747', '750', '756', '770', '774', '795', '801', '809', '811', '816', '841', '853', '863', '872', '884', '889', '892', '900', '914', '916', '917', '920', '922', '935', '943', '949', '971', '985', '991', '997', '1006', '1007', '1036', '1044', '1045', '1050', '1053', '1062', '1068', '1072', '1088', '1099', '1101', '1109', '1118', '1120', '1130', '1133', '1137', '1141', '1151', '1158', '1196', '1214',